In [1]:
import os
import pickle
import numpy as np
from keras.models import Sequential
import keras.layers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train, valid, test = pickle.load(open('dataset.pickle', 'rb'))
train = train.fillna(0)
valid = valid.fillna(0)
test = test.fillna(0)

In [3]:
feat_col = ['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']
x_train = train[feat_col]
y_train = train.iloc[:,-8:]
x_valid = valid[feat_col]
y_valid = valid.iloc[:,-8:]
x_test = test[feat_col]
y_test = test.iloc[:,-8:]

In [4]:
scaler = MinMaxScaler() #scale features between 0 and 1
x_train.loc[:,feat_col] = scaler.fit_transform(x_train)
#x_train = np.asarray(x_train).reshape(x_train.shape[0], 1, x_train.shape[1])
x_valid.loc[:,feat_col] = scaler.transform(x_valid)
x_test.loc[:,feat_col] = scaler.transform(x_test)
#x_test = scaler.fit_transform(x_test)
#x_test = np.asarray(x_test).reshape(x_test.shape[0], 1, x_test.shape[1])

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

In [16]:
x_train['raw_acc']

2                                               magnitude_stats            \
3                                                          mean       std   
filename                             timestamp                              
00EABED2-271D-49D8-B599-1D4A09240601 1444079161        0.312890  0.001823   
                                     1444079221        0.312905  0.002155   
                                     1444079281        0.312893  0.001894   
                                     1444079341        0.312908  0.001829   
                                     1444079431        0.313064  0.019445   
                                     1444079492        0.312985  0.002492   
                                     1444079552        0.312991  0.001810   
                                     1444079612        0.312999  0.000971   
                                     1444079672        0.313056  0.001091   
                                     1444079732        0.313022  0.001052   
                                     1444079792        0.313030  0.005113   
                                     1444079852        0.313012  0.001537   
                                     1444079912        0.312941  0.001199   
                                     1444079972        0.312960  0.001763   
                                     1444080032        0.312962  0.001020   
                                     1444080092        0.312987  0.001268   
                                     1444080152        0.313015  0.001970   
                                     1444080212        0.312818  0.001122   
                                     1444080272        0.312932  0.001096   
                                     1444080332        0.313110  0.019070   
                                     1444080392        0.313016  0.001298   
                                     1444080452        0.312586  0.015397   
                                     1444080512        0.314575  0.033957   
                                     1444080572        0.315439  0.074330   
                                     1444080632        0.313429  0.022239   
                                     1444080692        0.313062  0.000927   
                                     1444080752        0.312921  0.000698   
                                     1444080812        0.312607  0.000827   
                                     1444080872        0.313028  0.000661   
                                     1444080932        0.312975  0.002662   
...                                                         ...       ...   
5152A2DF-FAF3-4BA8-9CA9-E66B32671A53 1443584586        0.315951  0.015832   
                                     1443584646        0.315933  0.045668   
                                     1443584706        0.316002  0.017459   
                                     1443584766        0.313269  0.056931   
                                     1443584826        0.318932  0.073382   
                                     1443584886        0.332177  0.146432   
                                     1443584946        0.318503  0.026855   
                                     1443585006        0.318237  0.041333   
                                     1443585066        0.318368  0.050962   
                                     1443585126        0.316435  0.072113   
                                     1443585186        0.314405  0.015710   
                                     1443646032        0.319863  0.055576   
                                     1443646110        0.314979  0.042433   
                                     1443646170        0.323799  0.105716   
                                     1443646230        0.310639  0.001088   
                                     1443646290        0.320337  0.111923   
                                     1443646350        0.314863  0.049024   
                                     1443646410        0.331167  0.212491

In [5]:
# fit model on dataset
def fit_model(x_train, y_train, x_valid, y_valid):
    model = Sequential()
    #model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2]), bias_regularizer=L1L2(l1=0.01, l2=0.01)))
    #model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2])))
    #model.add(Dense(2000, activation='relu'))
    model.add(Dense(200, input_dim=x_train.shape[1], activation='relu'))
    model.add(Dropout(0.2, noise_shape=None, seed=None))
    model.add(BatchNormalization())
    model.add(Dense(50, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(y_train.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.fit([x_train], batch_size=20, y=y_train, verbose=1, validation_split=0.25, shuffle=True, epochs=1)
    model.fit([x_train], batch_size=100, y=y_train, verbose=1, validation_data=(x_valid, y_valid), shuffle=True, epochs=20)
    return model

In [6]:
def get_x_data(fields):
    train_data = x_train[fields]
    valid_data = x_valid[fields]
    test_data = x_test[fields]
    return (train_data, valid_data, test_data)

In [7]:
# create directory for models
dir_name = "nn_ensemble"
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
# fit and save models
target_fields = ['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']
for i in range(len(target_fields)):
    # fit model
    x_data = get_x_data(target_fields[i])
    train_x = x_data[0]
    valid_x = x_data[1]
    model = fit_model(train_x, y_train, valid_x, y_valid)
    # save model
    filename = dir_name + '/model_' + str(i + 1) + '.nn'
    model.save(filename)
    print('Saved %s' % filename)

Train on 298911 samples, validate on 30026 samples
Epoch 1/20
298911/298911 [==============================] - 26s 86us/step - loss: 1.3995 - acc: 0.4747 - val_loss: 1.3267 - val_acc: 0.5073
Epoch 2/20
298911/298911 [==============================] - 24s 79us/step - loss: 1.3395 - acc: 0.4944 - val_loss: 1.3738 - val_acc: 0.4999
Epoch 3/20
298911/298911 [==============================] - 25s 85us/step - loss: 1.3290 - acc: 0.4999 - val_loss: 1.3632 - val_acc: 0.5000
Epoch 4/20
298911/298911 [==============================] - 25s 85us/step - loss: 1.3203 - acc: 0.5052 - val_loss: 1.3626 - val_acc: 0.4543
Epoch 5/20
298911/298911 [==============================] - 26s 87us/step - loss: 1.3139 - acc: 0.5080 - val_loss: 1.3791 - val_acc: 0.4560
Epoch 6/20
298911/298911 [==============================] - 25s 84us/step - loss: 1.3082 - acc: 0.5109 - val_loss: 1.3433 - val_acc: 0.5024
Epoch 7/20
298911/298911 [==============================] - 25s 83us/step - loss: 1.2853 - acc: 0.5236 - val_

Epoch 7/20
298911/298911 [==============================] - 26s 86us/step - loss: 1.2953 - acc: 0.5010 - val_loss: 1.3191 - val_acc: 0.4528
Epoch 8/20
298911/298911 [==============================] - 26s 87us/step - loss: 1.2905 - acc: 0.5024 - val_loss: 1.3069 - val_acc: 0.4607
Epoch 9/20
298911/298911 [==============================] - 25s 85us/step - loss: 1.2888 - acc: 0.5023 - val_loss: 1.3035 - val_acc: 0.4565
Epoch 10/20
298911/298911 [==============================] - 25s 84us/step - loss: 1.2860 - acc: 0.5047 - val_loss: 1.2904 - val_acc: 0.5713
Epoch 11/20
298911/298911 [==============================] - 25s 85us/step - loss: 1.2859 - acc: 0.5042 - val_loss: 1.2896 - val_acc: 0.5668
Epoch 12/20
298911/298911 [==============================] - 26s 86us/step - loss: 1.2828 - acc: 0.5052 - val_loss: 1.3091 - val_acc: 0.4594
Epoch 13/20
298911/298911 [==============================] - 25s 85us/step - loss: 1.2802 - acc: 0.5054 - val_loss: 1.3063 - val_acc: 0.4607
Epoch 14/20
2989

298911/298911 [==============================] - 26s 86us/step - loss: 1.5333 - acc: 0.3783 - val_loss: 1.5907 - val_acc: 0.3475
Epoch 4/20
298911/298911 [==============================] - 25s 83us/step - loss: 1.5297 - acc: 0.3796 - val_loss: 1.5476 - val_acc: 0.3190
Epoch 5/20
298911/298911 [==============================] - 25s 85us/step - loss: 1.5257 - acc: 0.3843 - val_loss: 1.5486 - val_acc: 0.3731
Epoch 6/20
298911/298911 [==============================] - 26s 86us/step - loss: 1.5228 - acc: 0.3882 - val_loss: 1.5285 - val_acc: 0.3798
Epoch 7/20
298911/298911 [==============================] - 24s 81us/step - loss: 1.5210 - acc: 0.3898 - val_loss: 1.5407 - val_acc: 0.3967
Epoch 8/20
298911/298911 [==============================] - 24s 80us/step - loss: 1.5203 - acc: 0.3901 - val_loss: 1.5476 - val_acc: 0.3621
Epoch 9/20
298911/298911 [==============================] - 24s 80us/step - loss: 1.5184 - acc: 0.3917 - val_loss: 1.5316 - val_acc: 0.3969
Epoch 10/20
298911/298911 [====

298911/298911 [==============================] - 25s 85us/step - loss: 1.3692 - acc: 0.5052 - val_loss: 1.3729 - val_acc: 0.5407
Saved nn_ensemble/model_9.nn
Train on 298911 samples, validate on 30026 samples
Epoch 1/20
298911/298911 [==============================] - 27s 90us/step - loss: 1.2603 - acc: 0.5628 - val_loss: 1.2499 - val_acc: 0.5843
Epoch 2/20
298911/298911 [==============================] - 26s 86us/step - loss: 1.2149 - acc: 0.5704 - val_loss: 1.2476 - val_acc: 0.5815
Epoch 3/20
298911/298911 [==============================] - 26s 87us/step - loss: 1.2090 - acc: 0.5718 - val_loss: 1.2595 - val_acc: 0.5689
Epoch 4/20
298911/298911 [==============================] - 26s 86us/step - loss: 1.2049 - acc: 0.5724 - val_loss: 1.2712 - val_acc: 0.5703
Epoch 5/20
298911/298911 [==============================] - 26s 86us/step - loss: 1.2034 - acc: 0.5729 - val_loss: 1.2937 - val_acc: 0.5626
Epoch 6/20
298911/298911 [==============================] - 25s 84us/step - loss: 1.2014 - 

In [ ]:
target_fields = ['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']
x_data = get_x_data(target_fields[0])
print(x_data[0].shape)
print(len(target_fields))